In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import isnull, when, count, isnan
from pyspark.sql import functions as F
from pyspark.sql import types

# W261 Machine Learning at Scale | Spring 2022 Final Project
Courtney Smith | Hassan Saad  | Frances Leung |  March Saper

## 1. Introduction

#### Motivation
A 2019 report by the FAA found that the annual cost associated with airline delays was $33 billion - and increasing by $3 billion per year since 2016. The cost of delays in 2019 was slightly more than 13% of the total value of the US domestic airline industry (valued at approximately $250 billion pre-Covid).[[1]](https://www.statista.com/statistics/197680/total-operating-revenues-in-us-airline-industry-since-2004/)[[2]](https://www.faa.gov/data_research/aviation_data_statistics/media/cost_delay_estimates.pdf)   
For carriers, the costs come from anticipated and unanticipated delays. For anticipated delays, the costs are due to built-in schedule buffers; for unanticipated delays, the costs come from increased operational expenses from crew, fuel, and maintenance costs, as well as lost demand as a result of decreased customer trust.   
For customers, who end up absorbing just under 55% of the financial impacts, the costs result from lost time due to delays, cancellations, and missed flight connections, in addition to increased food and lodging expenses from more time away from home.[[2]](https://www.faa.gov/data_research/aviation_data_statistics/media/cost_delay_estimates.pdf)


Being able to predict delays in advance could alleviate some of these costs for both airlines and their client base. For example, anticipating delays ahead of time could allow passengers to reschedule connecting flights (or better yet, could allow airlines to do this for their passengers), which could prevent customer dissatisfaction and improve retention. This would also give airlines time to optimize the cost efficiency of any operational adjustments necessitated by the delay. Additionally, with more accurate delay prediction, airlines may be able to shrink their built-in schedule buffers and reduce the costs around anticipated delays.

#### Question Formulation

The objective of this project is to predict flight delays 2 hours before their planned departure time. A delay is defined as any departure that occurs at least 15 minutes after it was intentionally planned. 

According to the FAA, 69% of all delayed flights are a result of weather complications and the cascading effect of impacted flights. In 2013 alone the portion of delay due to weather was nearly 10 million cumulative minutes (equal to ~19 years).[[3]](https://www.faa.gov/nextgen/programs/weather/faq/) To achieve our objective requires incorporating weather variables into our feature set.

Since our target variable will be a binary indicator, we will need to use a classifier such as Logistic Regression or Decision Trees. Logistic regression will scale well but we'll need to implement regularization in order to avoid overfitting. Decision Trees can be complemented with Gradient Boosting to improve performance, however the use of gradients can be very compute intensive since it requires the decision trees to be built sequentially and can't be parallelized.

#### Data Sets 
There are two datasets used in our model: flight data and weather data. The flight data came from the US Department of Transportation and contains flight records from 2015-2019.[[4]](https://www.transtats.bts.gov/homepage.asp) The weather data comes from the National Oceanic and Atmospheric Administration and contains 3-hourly quality-controlled weather observations from major airport weather stations and first- and second-order National Weather Service stations.[[5]](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00679)

#### Chosen Performance Metrics
We used the following metrics when evaluating our delay predictions:  
- Recall, which represents the fraction of true positive labels that we capture with our model: \\(\frac{TP}{TP+FN}\\). High recall is achieved with low rates of false negatives.   
- Precision, which represents the fraction of positive labels that are correctly predicted: \\(\frac{TP}{TP+FP}\\). High precision is achieved with low rates of false positives.   

We used an \\(F_{beta}\\) score, specifically \\(F_{2}\\), to weight recall more heavily when evaluating our model performance. The equations for this are as follows:

\\( F_{\beta} = \frac{(1+ \beta^2)(Precision \cdot Recall)}{\beta^2 \cdot Precision + Recall} \\)

\\( F_{2} = \frac{(1+ 2^2)\cdot Precision  \cdot Recall}{2^2 \cdot Precision + Recall} = \frac{5 \cdot Precision \cdot Recall}{4 \cdot Precision + Recall} = \frac{5 \cdot TP}{5\cdot TP + 4 \cdot FN + FP} \\)

Using F2 as our primary metric will put more emphasis on acheiving higher recall to prioritize reducing false negatives, which carry more financial and operational costs, over false positives.

### Baseline

We had two stages of baseline models that we "built" before we created a more sophisticated pipeline. The first was a simple majority class guess. Given the label distribution, this resulted in a ~76.5% accuracy. Given that there is a cost associated with false positives (E.g. unnecessary lodging or food costs for customers and staff), we needed to create a model that could capture a larger share of the negative class labels as well.

After we joined the weather and airline tables for Q1, we built a Random Forest model with the ~161k records. This allowed us gauge how much predictive power the weather data contained. The first iteration with zero hyperparameter tuning gave us an accuracy of about ~79.5%, a minor improvement over the majority class guess baseline. This confirmed that our model would be much more reliant on time series and network effect features than on weather data.

### Notebook Organization
This main notebook documents the methodology and results of the flight delay prediction machine learning models built. A list of supplementary notebooks can be found in section 9, References, at the very end of the document. The respective supplementary notebook that offer more details on the work completed for each section of this document are linked at the beginning of each section.

## 2. Data Methodology

Supplementary notebook related to data join can be found here [[NB1 | Airline Weather Full Join]](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102379377/command/1858507102379379).

#### Data Overview

The key data sources used include those related to flights and weather. The flights data represents a subset of the passenger flight's on-time performance data retrieved from the TransStats data collection available from the US Department of Transportation (DOT)[[4]](https://www.transtats.bts.gov/homepage.asp). The weather data was retrieved from the National Oceanic and Atmospheric Administration repository[[5]](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00679)[[7]](https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf). A global airport database [[6]](https://www.partow.net/miscellaneous/airportdatabase/index.html) was also leveraged to retrieved the airport locations. Below are some details on the datasets:

**Flights data**
1. Flight information from 2015 to 2019
2. 31,746,841 x 109 dataframe
3. Features include those related to the following categories:
  - Time Period
  - Airline
  - Origin
  - Destination
  - Departure Performance
  - Arrival Performance
  - Cancellation and Diversions
  - Flight Summaries
  - Cause of Delay
  - Gate Return Information at Origin Airport
  - Diverted Airport Information

**Weather data**
1. Weather data from 2015 to 2019
2. 630,904,436 x 177 dataframe
3. Features include those related to the following categories:
  - Weather stations
  - Wind observations
  - Liquid precipitation
  - Sky condition
  - Visibility observation
  - Air temperature
  - Snow depth
  - Snow accumulation
  - Cloud and solar data
  
**Airport data**
1. Global airport database
2. Features include:
  - ICAO code
  - IATA code
  - Name
  - Country
  - City
  - Latitude-Longitude position
  - Altitude

#### Building the Joined Dataset

To join the weather and flights tables, we leveraged the location of each airport and associated it with the closet weather station. Since the flights table does not contain location data (longitude/latitude), we used an intermediate table called the Global Airport Database [[6]](https://www.partow.net/miscellaneous/airportdatabase/index.html) (which we will refer to as `airport`) to join it to the weather data. The next steps are as follows:

1. We cross joined the unique weather stations and the `airport` data (resulting in a table with every airport matched to every weather station), then add a column representing the haversine distance between them. Group by airport and aggregate to find the minimum haversine distance. Merge this new table with `df_airlines` to add a feature representing the closest weather station for the corresponding flight departure.

2. Before associating the weather and airline data, the airport times needed to be converted into UTC. Another intermediate table from OpenFlights.org [[8]](https://openflights.org/data.html) that contains each airport acronym as well as its offset from UTC time was used. We leveraged the `to_utc_timestamp()` pyspark method, which took the time and offset values to create a UTC time feature. 

3. Taking the `df_airlines` table (with the new weather station ID column) as the left table and `df_weather` as the right, a left join was performed using the 3-character airport acronym referred to as the ICAO code as the key. We then created a column that represented the time delta between the weather data and the departure time, and aggregated to find the minimum time delta (but still > 2 hrs). 

There was no additional constraint placed on the join, but if we were to repeat the process we would also join on time ranges so as to limit the final aggregation complexity. The final join took just under 7 hours. 

**Join Process Flowchart:**



<img src="files/shared_uploads/hassansaadca@berkeley.edu/DataJoins.png" width="1000"/>

## 3. Exploratory Data Analysis

Supplementary notebook related to EDA can be found here:
- [[NB2 | EDA Part 1 - Airline and Weather Data]](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102378189/command/1858507102378190)
- [[NB3 | EDA Part 2 - Weather and Time-Related Data]](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102431022/command/1858507102431092)

The reason for two EDA notebooks:
While our initial EDA served to start understanding the data, it also helped to continue creating plots and charts as we built more sophisticated features and evaluated the their effectiveness while tuning our models.

The exploratory data analysis (EDA) began with examining the shape and schema of the dataframes. We looked at delays as a function of the following variable groups:

1. Weather conditions at the origin airport (E.g. temperature, visibility, precipitation)
2. Flight info features (E.g. origin and destination airports, carrier)
3. Time series-related features (E.g. quarter, month, day of the week) to check for cycles or seasonality

Based on the results of our EDA, we concluded that weather variables were most useful on an aggregate level: grouped by airport and averaged over the previous week of readings. We found optimal thresholds for binarizing air temperature and visibility to maximize information gain from those variables.  

Time series elements appeared to play a role in the frequency of delays so we opted to keep those features. 

Features related to flight info were kept as categorical variables. We also found that incorporating a time element (ex. % of delayed flights per carrier over the previous 24 hours) created additional value from these variables, so we generated time-dependent delay averages in our feature engineering phase.

In [0]:
#DATA SOURCE SETUP

blob_container = "team20fp" # The name of your container created in https://portal.azure.com
storage_account = "w261fp" # The name of your Storage account created in https://portal.azure.com
secret_scope = "team20scope" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "team20key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

# Read raw airlines data
df_airlines = spark.read.parquet("/mnt/mids-w261/datasets_final_project/parquet_airlines_data/*").cache()
# Read raw weather data
df_weather = spark.read.parquet("/mnt/mids-w261/datasets_final_project/weather_data/*").cache()
# Read fully joined data
df = spark.read.parquet(f"{blob_url}/airlines_weather_full")

#### 3.1 Weather Feature Analysis
In our initial analysis of the weather data, we looked at the distribution of each column and the count of populated records. We dropped columns with very sparse data and forward filled the remaining ones. We focused on the following weather features:
* rain
* wind
* visibility
* temperature
* atmospheric pressure

We started with rain and quickly noticed that there was not a clear relationship with delays: the proportion of delayed flights did not consistently increase as the amount of rain increased. 


<img src ='files/shared_uploads/hassansaadca@berkeley.edu/rain.png' width="1000"/>

We noticed that wind speed alone did not have a strong relationship with departure delays. We plotted a similar bar chart to the one above, and while there is a positive correlation, the differentiation is not as significant as expected. It's only in combination with direction (i.e. a crosswind or tailwind) that wind is a significant factor in departure timing. Unfortunately, the wind direction data was measured in degrees relative to the weather station, so we did not have information on the direction relative to the airport runway. The chart below shows the difference in wind direction vs minutes of departure delay for airports ATL and ORD. We can see that the ATL experiences more delays when winds are in the directions between 100 and 250 degrees (relative to the closest weather station to ATL), where ORD experiences more when the direction is bewteen 250 to 50 degrees (again, relative to the station closest to ORD).


<img src ='files/shared_uploads/hassansaadca@berkeley.edu/wind_speed.png' width="1000"/>



<img src ='files/shared_uploads/hassansaadca@berkeley.edu/wind_dir.png' width="1000"/>

We saw that visibility was strongly correlated with delay for extreme values - the below table shows a binary split where ~10% of the observations fall into the low visiblity group. The ideal value was found by iterating through threshold values and selecting the value that maximized the sum of the differences. On average, there was a 13 point difference in the percentage of delayed flights between the two groups. For some airports, like ORD, this difference was much larger.

|ORIGIN|LOW_VIS_IND|DEL_PCT|
|---|---|---|
|ATL   |1 |0.297|
|ATL   |0 |0.158|
|ORD   |1 |0.521|
|ORD   |0 |0.283|

We took a similar approach with temperature. The extreme values had a 25 point higher rate of delayed flights. Again, this difference varied between airports (shown below, ATL had no readings in the low temperature group) and the optimal threshold was found by maximizing the sum of the differences.

|ORIGIN|LOW_TMP_IND|DEL_PCT|
|---|---|---|
|ATL   |1 |n/a|
|ATL   |0 |0.175|
|ORD   |1 |0.472|
|ORD   |0 |0.273|

We saw that only extreme values of these weather features (which represent a very small portion of the data distribution) had a strong correlation with positive class labels (`DEP_DEL15`). This led to our decision to create features for rolling time-based averages and binary indicators for extreme values.

#### 3.2 Airport and Airline Features

We figured that there would be a lot of insight to be gained from analyzing delays when grouping by airline or airport. For example:
* some airports are smaller and therefore use smaller planes, which are more susceptible to weather changes
* airlines can differ in terms of the operational resources they have, which can make smaller ones more susceptible to delay

Below, we demonstrate some of the EDA we did related to the second bullet (i.e. airline-related statistics)

##### Carriers with the highest number of flights:

In [0]:
df_carrier_flights_vol=spark.read.parquet(f"{blob_url}/df_carrier_flights_vol")
display(df_carrier_flights_vol)

OP_UNIQUE_CARRIER,count
WN,6522590
DL,4497788
AA,4372346
OO,2839124
UA,2831411
EV,1520254
B6,1435635
AS,1036679
NK,783119
MQ,768383


##### Proportion of Flights Delayed by Carrier

By examining the proportion of flight delays by carrier, we can see that overall, it is roughly a 80/20 split of on-time vs. delayed (i.e. > 15 minutes delay) for the majority of carriers. 

Particular carriers do look to be more prone to delays, however. The top 4 carriers that have the highest proportion of delay at 25%, 24%, 21% and 21% respectively are as follows:
 1. B6 (JetBlue Airways Corporation)
 2. F9 (Frontier Airlines)
 3. WN (Southwest Airlines Co.)
 4. VX (Virgin America)

In [0]:
df_flight_delay_proportion2=spark.read.parquet(f"{blob_url}/df_flight_delay_proportion2")
display(df_flight_delay_proportion2)

OP_UNIQUE_CARRIER,DEP_DEL15,count
WN,0.0,5074385
DL,0.0,3829688
AA,0.0,3547884
OO,0.0,2332565
UA,0.0,2290222
WN,1.0,1363239
EV,0.0,1207449
B6,0.0,1056479
AS,0.0,896641
AA,1.0,768563


#### 3.3 Time-related Feature Analysis:

Time elements including month and day of week  were examined to identify if there are correlation with flight delays. 
- Month: Summer months including Jun, Jul, Aug and Dec have a higher proportion of flight delays. This appear to coorelates with seasonality and holiday peaks
- Day of Week: Mon, Thu and Fri appear to have a higher proportion of flight delays
**Number of flights delayed by Month**

We figured there would be some correlation between the month and the number of delays, even if we didn't know exactly how prior to starting this part of the EDA process. We plotted the number of delays per month and discovered that June, July, and August had the most, and that September through November had the fewest. We validated this by seeing if the trend was consistent across airports, which it was. As seen in the chart below, the bar charts across 3 different airports show similar characteristics in terms of the month and the number of delays.

<img src ='files/shared_uploads/hassansaadca@berkeley.edu/months.png' width="1000"/>


We also did some analysis on the correlation between day of week by selected airports . We grouped the Q1 data by airport prior to plotting a bar chart (% of flights delayed as a result of the weekday) and noticed that there was definitely a correlation between the day of week and the likelihood of a delay greater than 15 minutes. Again, we made sure this was a consistent trend by grouping the plots by airport (though in this case the blue plot represents both airports ATL and ORD, and the yellow and red charts show them separated out). Monday had a significantly higher percentage of delayed flights for both airports ATL and ORD, and Saturday had the fewest. 


**% of flight delay by day of week overall and for selected airports (2015 Q1 data)**

<img src ='files/shared_uploads/marchsaper@berkeley.edu/bar1.png'/>

During the EDA process, we discovered that the most important features would be related to time-series characteristics within our dataset. While it seemed very trivial initially, we created a scatterplot of the relationship between departure and arrival delay minutes (image below). We noticed that there was almost a 1 to 1 relationship between these variables (`ARR_DELAY` as a function of `DEP_DELAY`). This indicates that very few of the flights that experience departure delays greater than 15 minutes are able to arrive on time (i.e. within < 15 mins of planned arrival). Taking this a step further, given that airlines likely try to maximize the number of flights a plane takes in a day, the likelihood of a late departure affecting the next one for the same plane is quite high.


<img src ='files/shared_uploads/hassansaadca@berkeley.edu/dva.png' width="500"/>

Another way we verified this was not necessarily through charts, but just by sorting data by plane (`TAIL_NUM`) and departure time. The image below shows an example of the fact that it's difficult for a plane to get back on its originally-planned flight schedule after a single delay early in the day (using plane number N35407 on 12/18/2018 as an example).

<img src ='files/shared_uploads/hassansaadca@berkeley.edu/n35407.png' width="500"/>



This was great motivation to start building time-series and network related features, which we discuss in the following section.

#### Missing Values in Labels and Features
Many of the weather stations produced updated reports only when the values had changed, otherwise they were left blank or filled with a null value (E.g. 999, 99999). To address this, we forward-filled our selected weather variables.

## 4. Feature Engineering

Supplementary notebook related to feature engineering can be found here [[NB4 | Feature Engineering]](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102383058/command/1858507102383069).

%md
Our features can be grouped into 3 different categories:
* The first relates to data tied to the current flight record. These mostly consist of features for which there was minimal or no transformation work done to extract the underlying data. The few that did require minor processing are explained below.
  * `ORIGIN`
  * `ORIG_FPD`: The number of flights per day leaving from the origin airport
  * `DEST_FPD`: The number of flights per day arriving at the destination airport
  * `DEST`
  * `MONTH`
  * `DAY_OF_WEEK` 
  * `QUARTER`
  * `OP_CARRIER`
  * `DISTANCE_GROUP`
  * `ORIG_DEST`: A feature that concatenates the string values of `ORIGIN` and `DEST`, to represent the flight route
  * `VIS`
  * `AIR_TMP`
  
  
Most of the columns in the weather table included several data points each, so it was necessary to identify which portion was of interest, then extract it as an integer rather than a string value. The below code snippet shows a UDF that we relied on a lot throughout the feature engineering process, as well as a sample of how it was applied.

In [0]:
#create a user defined function that splits the string data and returns the item corresponding to the relevant index (idx)
def extract(block, idx):
    try:
        parts = block.split(',')
        return(int(parts[idx]))
    except:
        return None


extractUDF= F.udf(lambda column, index: extract(column, index), types.IntegerType()) 


#extract wind direction and speed from WND
#" rain levels, ceiling heigh, etc.
df = df.withColumn('WND_DIR', extractUDF('WND', F.lit(0)))\
          .withColumn('WND_SPEED', extractUDF('WND', F.lit(3)))\
          .withColumn('AA2', extractUDF('AA1', F.lit(1)))\
          .withColumn('AA1', extractUDF('AA1', F.lit(0)))

* The second group consists of time-dependent network effect-related features 
  * `PRE_FL_WINDOW`: The number of hours between the scheduled arrival of the previous flight and the scheduled departure of the current one (by tail number)
  * `PRIOR_ARR_DEL`: A binary indicator to represent whether the previous flight for the same plane was delayed on arrival (conditional on `PRE_FL_WINDOW`>2).
  * `PRIOR_DEP_DEL`: A binary indicator to represent whether the previous flight for the same plane was delayed on departure (conditional on `PRE_FL_WINDOW`>2).
  
  These features model how a plane's delay at one airport propagates to the same plane at the next airport. They essentially treat each plane's flight path as a network that resets on a daily basis, and they model whether or not a value at one node of the network (an airport) transfers to the next node. There are two caveats related to these features:
  * The first is that we had to make sure that we weren't considering records for which the scheduled arrival of the previous flight was less than 2 hours before the current one's departure. We used the `PRE_FL_WINDOW` feature to output an intermediate column, which represented whether this time window was between 2 and 12 hours with a binary value.  `PRIOR_ARR_DEL` and `PRIOR_DEP_DEL` contain a 1 if both the previous flight for the same plane was delayed and if the intermediate column also contained a 1. 0 means that the time frame is between 2 and 12 hours but the previous flight was not delayed, and a -1 means the time frame was either greater than 12 hours or fewer than 2.
  * The next caveat is that, in a practical setting, these features prevent us from being able to run predictions unless our training sets are 100% up to date, which means we have to load and join datatables very regularly.

* The third group of features consists of aggregate time-dependent statistics related to both weather and flights:
  * `PCT_CARRIER_DEL`: percentage of flights that were delayed within 26 and 2 hours prior to the current flight's departure, grouped by airline
  * `PCT_ROUTE_DEL`: percentage of flights that were delayed within 26 and 2 hours prior to the current flight's departure, grouped by route
  * `PCT_ORIGIN_DEL`: percentage of flights that were delayed within 26 and 2 hours prior to the current flight's departure, grouped by origin airport
  * `PCT_DEST_DEL`: percentage of flights that were delayed within 26 and 2 hours prior to the current flight's departure, grouped by destination airport
  * Weekly cumulative averages of weather statistics, grouped by airport:
    * `CUMAVG_WND_DIR_WEEKLY`
    * `CUMAVG_DEW_WEEKLY`
    * `CUMAVG_VIS_WEEKLY`
    * `CUMAVG_SLP_WEEKLY`
    * `CUMAVG_WND_SPEED_WEEKLY`
    * `CUMAVG_CEIL_HEIGHT_WEEKLY`
    * `CUMAVG_AIR_TMP_WEEKLY`
  * Weekly cumulative average of delay statistics, grouped by tail number (plane):
    * `CUMAVG_DEP_DELAY_WEEKLY`
    * `CUMAVG_DEP_DEL15_WEEKLY`
    * `CUMAVG_ARR_DELAY_WEEKLY`
    * `CUMAVG_ARR_DEL15_WEEKLY`

Since we had to create so many aggregate statistics, we created a function that took in 4 arguments:
* the original dataframe 
* the columns for which we wanted to create the aggregation
* the timeframe across which we wanted to aggregate
* a boolean indicator representing whether or not the columns were related to weather features (if not, we had to implement a time shift to make sure a record's own value wasn't included in the aggregate statistic)

This is shown below (excluding the additional help function that adds a 1-record shift depending on the type of feature we're passing in)

In [0]:
#add column that is a cumulative average of another. default partition is to take weekly average
def add_cumulative_average(dataframe, columns = None, weather_features = False, timeblock = 'weekly'):
    
    '''
    in: dataframe, features, boolean indicating if weather features, timeframe across which to aggregate
    out: new dataframe with original features + aggregated
    '''
    
    #identify time frame across which to aggregate
    if timeblock == 'weekly':
        time_window  = 'WEEK'
        dataframe = dataframe.withColumn(time_window, F.weekofyear('UTC_DEP_TIME'))
    elif timeblock == 'daily':
        time_window = 'DATE'
        dataframe = dataframe.withColumn(time_window, F.to_date('UTC_DEP_TIME'))
    else:
        print('INVALID TIMEFRAME PARAMETER. RETURNING ORIGINAL DATAFRAME')
        return dataframe
    
    #create partition based on weather features (airport partition) or non-weather features (tail number partition)
    if weather_features:
        window_partition = ['ORIGIN']
        order = ['ORIGIN','UTC_DEP_TIME']
    else:
        window_partition = ['TAIL_NUM']
        order = ['TAIL_NUM','UTC_DEP_TIME']
        #shift records by 1 if not related to weather (make sure record is not included in its own cumulative statistic)
        dataframe, shifted_columns = add_shift(dataframe, columns = columns)
    
    window = W.partitionBy(window_partition + [time_window]).orderBy(order)
    
    #get aggregate, but ignore 999 values if weather feature
    if weather_features:
        for i in columns:
            feat = f'CUMAVG_{i}_{timeblock.upper()}'
            dataframe = dataframe.withColumn(feat, F.when((col(i)!= 999) & (col(i) != 9999), F.sum(i).over(window)/F.count(i).over(window)))\
                                 .fillna(0, subset = [feat])
    else:
        for i in shifted_columns:
            feat = f'CUMAVG_{i[:-6]}_{timeblock.upper()}'
            dataframe = dataframe.withColumn(feat, F.sum(i).over(window)/F.count(i).over(window))\
                                  .fillna(0, subset = [feat])\
                                  .drop(i)
        
#     dataframe = dataframe.drop(time_window)
    
    return(dataframe)

The below images shows a sample of the cumulative weather statistics for Abilene Regional Airport. While the cumulative weather statistics are all partitioned by airport, the cumulative delay statistics are grouped by tail number.
    
<img src ='files/shared_uploads/hassansaadca@berkeley.edu/cumweather.png' width="1000"/>

These features help us represent general "health" metrics related to specific airports and airlines. They also help make sure that our models are less susceptible to outliers, since weather and delay anomalies are dampened when aggregated across a week of data for each aiport.

## 5. Model Implementation

Supplementary notebook related to model implementaiton/machine learning pipeline could be found here:
- [[NB5 | Machine Learning Pipeline]](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574710015602/command/4070574710015603)
- [[NB8 | Final Pipeline Test]](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102387041/command/1858507102400388)

There were a few post processing tasks that we had to complete depending on the feature type and the model that we wanted to use. Where with scikit-learn these processes would be part of the feature engineering, they are implemented in the pipeline generation portion when building ML models in Pyspark. 

### In-Pipeline Feature Processing

#### Standard Scaling
The simplest of these transformations was to standard scale the continuous features, though this was only required for logistic regression. Considering the fact that Pyspark implements a gradient descent algorithm under the hood of its logistic regression model, standard scaling will make this loss function more symmetrical and allow for faster convergence when learning the weight vector. 


#### Bucketing
The majority of the features in group 1 above consist of discrete identifiers, and some of them have very high numbers of distinct features. We realized that as a result of this, implementing one-hot encoding for these features could yield too many features and lead to overfitting. Our solution was to create the buckets for the discrete variables that had high variance or had very non-uniform distribution. This includes features such as `ORIGIN`,`VIS` , `AIR_TMP`, `ORIG_FPD` (max flights per day from the origin), etc. For other category 1 features such as day of week or quarter, bucketing was not implemented since there are few distinct values.

### 5-Fold Cross-Validation

Before building a machine learning pipeline, we split our data into training and test set blocks. We followed the recommended approach, and we first sorted the final joined airline/weather table by departure date. We then split it into 5 chronological folds with no overlap between each one. Each fold was then separated into a train and test set (with an 80% / 20% split, respectively).

In order to save time running and testing our models, these folds were each saved in our blob in a separate folder, and we created a simple function to load them all as a list of dataframes at the beginning of every modeling/pipeline notebook.

### Data Balancing

The class distribution for our target labels was about 23%, so we created a function to upsample or downsample the train records within our pipeline. The downsample process involved taking a random sample without replacement from the negative class labels corresponding to the number of positive labels for each fold's train set. The upsample process took a random sample with replacement from the positive class labels until it matched the number of negative class labels. We opted for downsampling when training the models because it does not create repeat observations and it results in a smaller training set which sped up the grid search process. 

The code for the balancing can be found in the `balance_trainset` function in our pipeline notebook.

### Machine Learning Pipeline Flowchart
<img src="files/shared_uploads/hassansaadca@berkeley.edu/pipeline_final2.png" width="1000"/>

## 6. Algorithms and HyperParameter Tuning

Supplementary notebooks on algorithmes and hyperparameter tuning could be found here:
- [[NB6 | Decision Tree and Gradient Boost Toy]](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102378262/command/1858507102378263)
- [[NB7 | Logistic Regression Toy]](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102371728/command/1858507102371729)


Note: The above notebooks break down how each of the algorithms works. For Gradient Boost and XGBoost, we built a decision tree notebook that shows how node purity is calculated and how decisions are made based on this.

### Logistic Regression Model


The logistic regression model took about 9 minutes to train for a single set of hyperparameters:
* a regularization term \\(\lambda\\)
* an elastic net parameter \\(\alpha\\), which determines what proportion(s) of L1 and L2 regularization we will use

Our regularization cost component is represented by:

\\(\alpha (\lambda \times \lvert \bold{w} \rvert _{1}) + (1- \alpha)(\frac{\lambda}{2} \times \lvert \bold{w} \rvert _{2}^{2})    \\) 

such that when \\(\alpha =1\\), we end up with pure Lasso regularization, and \\(\alpha =0\\) will give us only Ridge regression.

We built the regression model in tandem with a baseline gradient boost model, and we quickly noticed that it was outperformed even before we started tuning hyperparameters. It yielded the following metrics with \\(\lambda = 0.1\\) and \\(\alpha = 0.8\\):

Limited Feature Set:
|Metric|Score|
|---|---|
|F2  |     0.4414|
|Precision | 0.2801|
|Recall  |  0.5449|

Full Feature Set:
|Metric|Score|
|---|---|
|F2  |     0.5121|
|Precision | 0.3212|
|Recall  |  0.6155|
                                    

The Gradient Boost model scored higher, so we shifted our focus to that instead.

### Gradient Boost (GB) Decision Tree

After deciding to go with a tree-based approach rather than logistic regression, we started tuning hyperparameters for a gradient boosted tree model. We used grid search to shuffle through four different parameters:

* `maxDepth` : The maximum depth of the tree trained at each iteration of the GB process
* `maxBins` : The maximum number of bins/buckets that a feature could be divided into in order to select a split point for a leaf
* `maxIter`: Number of sequential trees trained
* `stepSize`: The learning rate which was used to update the current tree in the model ( \\(\lambda\\) below)

\\(\hat{f}(x) \leftarrow \hat{f}(x) + \lambda \times \gamma \\) where lambda represents our learning rate and gamma represents an aggregate calculation of our residuals for each leaf node (refer to our toy algorithm notebook to see how we calculate \\(\gamma\\))


An example dictionary of some of these hyperparameters is noted below along with the distribution of F2 scores, and Precision/ Recall Values. 


Sample hyperparameters:

```
param_dict2 = {'gbt':{'maxDepth':[5,8],
                      'maxBins':[200,400],
                      'maxIter':[4,6],
                      'stepSize':[0.1, 0.3]}}
```

Distribution of corresponding evaluation metrics (Again, note that even our poorest F2 score is higher than the one we got for our logistic regression limited feature set baseline):

|`maxDepth`|`maxBins`|`maxIter`|`stepSize`| F2	| Precision |	Recall |
|---|---|---|---|---|---|---|
|5|200|4|0.1| 0.479730	|0.280828	|0.583485|
|5|200|4|0.3|0.491416	| 0.283016|	0.602408|
|5|200|6|0.1|0.481325	| 0.284846|	0.581790|
|5|200|6|0.3|0.489853	| 0.286931|	0.595850|
|5|400|4|0.1|0.477934	| 0.280667|	0.579977|
|5|400|4|0.3|0.494776	| 0.279434|	0.613160|
|5|400|6|0.1|0.484321	| 0.282778|	0.589342|
|5|400|6|0.3|0.496633	| 0.283825|	0.611646|
|8|200|4|0.1|0.482569	| 0.286050|	0.582692|
|8|200|4|0.3|0.488852	| 0.287550|	0.593580|
|8|200|6|0.1|0.487133	| 0.285579|	0.592045|
|8|200|6|0.3|0.484342	| 0.285992|	0.586366|
|8|400|4|0.1|0.484365	| 0.285452|	0.586646|
|8|400|4|0.3|0.487800	| 0.283730|	0.595172|
|8|400|6|0.1|0.485674	| 0.286694|	0.588105|
|8|400|6|0.3|0.479932	| 0.288852|	0.575726|


The above scores represent the early stages of our GB model pipeline which used a limited featureset (\~10 features, only features in categories 1 and 2 in our feature engineering section). 


After building our finalized featureset, we did another round of CV validation (a process that required us to run code overnight for \~10 hours), which yielded much higher F2 scores, the best of which is highlighted below.
```

gbt_paramdict :{'maxDepth':[6,9],
                      'maxBins':[250,300],
                      'maxIter':[6],
                      'stepSize':[0.2]}}
                      
```

|`maxDepth`|`maxBins`|`maxIter`|`stepSize`| F2	| Precision |	Recall |
|---|---|---|---|---|---|---|
|6|250|6|0.2|0.573545	|0.319713	|0.725539|
|6|300|6|6.2|0.574780	| 0.318967|	0.728463|
|9|250|6|0.2|0.576526	| 0.324444|	0.723345|
|**9**|**300**|**6**|**6.2**|**0.578219**	| **0.323277**|	**0.727845**|**

### XGBoost

XGBoost builds on gradient boost, this time implementing regularization by adding the following to the loss component. 

\\((\alpha \times (\lvert \bold{w} \rvert _{1})) + (\frac{\lambda}{2} \times \lvert \bold{w} \rvert _{2}^{2})    \\) 

where \\(\alpha\\) adjusts the L1/ Lasso regularization and \\(\lambda\\) adjusts the L2/ Ridge Regularization.

These regularization parameters correspond to `reg_alpha` and `reg_lambda` in the dictionary below.

We also adjusted two other parameters:

* `gamma`: The minimum loss reduction that is required in order to separate a leaf node into further splits (refer to the gamma in our GB discussion above, as well as our toy algorithm for more information on this)
* `learning_rate`: The proportion of the newest tree that will get added to the original one with each iteration.



Our first grid search included the following hyperparameters and yielded the results below (note that only the parameters that change are included in the table).

```
param_dict5 = {'xgb':{#booster = params['booster'],
                                'max_depth':[6],
                                'n_estimators':[150],
                                'reg_lambda':[0.5,1],
                                'reg_alpha':[0.1,0.5],
                                'tree_method':['hist'],
                                'objective':['binary:logistic'],
                                'base_score':[0.5],
                                'gamma':[0,0.05],
                                #'scale_pos_weight':[1],
                                'min_child_weight':[1.5],
                                #'max_delta_step':[0.7],
                                'max_bin': [100],
                                'learning_rate' : [0.2,0.3]}}                
```


|`reg_lambda`|`reg_alpha`|`gamma`|`learning_rate`|F2	| Precision	|Recall|
|---|---|---|---|---        | ---       | ---       |
|0.5|0.1|0  |0.2|0.584254   |	0.330522|	0.734255|
|0.5|0.1|0  |0.3|0.584311   |   0.329345|   0.735267|
|0.5|0.1|0.5|0.2|0.584371   |	0.330502|	0.734658|
|0.5|0.1|0.5|0.3|0.584274   |	0.329622|	0.734908|
|0.5|0.5|0  |0.2|0.584532   |	0.330556|	0.734892|
|0.5|0.5|0  |0.3|0.581830   |	0.329981|   0.729399|
|0.5|0.5|0.5|0.2|0.584341   |	0.330519|   0.734371|
|0.5|0.5|0.5|0.3|0.581927   |	0.329923|   0.729943|
|1  |0.1|0  |0.2|0.584572   |	0.330167|   0.735825|
|1  |0.1|0  |0.3|0.583468   |	0.329762|   0.733065|
|**1**  |**0.1**|**0.5**|**0.2**|**0.584771**   |	**0.329970**|   **0.736146**|
|1  |0.1|0.5|0.3|0.582550   |	0.330116|   0.730703|
|1  |0.5|0  |0.2|0.584034   |	0.330864|   0.733953|
|1  |0.5|0  |0.3|0.582215   |	0.330246|   0.730497|
|1  |0.5|0.5|0.2|0.584160   |	0.330388|   0.735136|
|1  |0.5|0.5|0.3|0.582574   |	0.329910|   0.731376|

The evaluation metrics don't fluctuate much between different parameter combinations, but the XGBoost model consistently improves performance over GB. The XGBoost cross validation process was quite slow, and the 16 parameter combinations tested above took ~19 hours. After this search, we took the parameters with the highest score (bold row above), and implemented additional tuning by alternating the lasso regression and binning value (again, the binning value indicates how many different buckets each feature can be partitioned into while deciding how to split a node). 


```
param_dict6 = {'xgb':{#booster = params['booster'],
                                'max_depth':[6],
                                'n_estimators':[150],
                                'reg_lambda':[1],
                                'reg_alpha':[0.1,0.2],
                                'tree_method':['hist'],
                                'objective':['binary:logistic'],
                                'base_score':[0.5],
                                'gamma':[0.05],
                                'min_child_weight':[1.5],
                                'max_bin': [50,100],
                                'learning_rate' : [0.2]}}

```

|`reg_lambda`|`reg_alpha`|`gamma`|`learning_rate`|`max_bin`|F2	| Precision	|Recall|
|---|---|---|---|---|---        | ---       | ---       |
|1|0.1|0.5  |0.1|50|0.583930   |	0.330424|	0.730531|
|1|0.1|0.5  |0.1|100|0.583930   |   0.330424|   0.729170|
|**1**|**0.2**|**0.5**|**0.2**|**50**|**0.584068**   |	**0.331052**|	**0.729170**|
|1|0.2|0.5|0.2|100|0.584068   |	0.331052|	0.729170|

The new alpha value which slightly increases the amount of regularization was more effective, and the binning quantity had no effect across the 4 tests. We selected the highlighted parameters above for our test on the full dataset.

## 7. Conclusions

Supplementary notebook on evaluation of model performance in the form of a gap analysis could be found here [[NB9 | Gap Analysis]](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1731553363147681/command/1731553363147686).

Our final model was evaluated on a holdout dataset from 2019 and yielded an F2 score of 0.6014.  

|Metric|Score|
|---|---|
|F2      | 0.6014|
|Precision|0.3275|
|Recall  | 0.7604|
|Accuracy|0.6617|

We optimized for a low false negative rate, and the model managed to capture 76% of true positives. On the other hand, this optimization caused our model to have a high rate of false positives - only 33% of predicted positives were true positives, and 36% of true negatives were labeled as positives. We tested a version of the model using a parameter to scale for class imbalances, and although this greatly increased our recall, it further lowered precision and accuracy, so we opted to keep this version. The effect of the class imbalances is seen throughout the gap analysis: segments with highly imbalanced classes had lower F2 scores.

<img src="files/shared_uploads/marchsaper@berkeley.edu/confusion_matrix-1.png"/>

From a business perspective, these results are better than implementing a majority class model (i.e. our baseline). False negatives likely have the highest associated cost of all the prediction outcomes, and a majority class model in our case would obviously capture none of these costs. At the same time, a model that is not conservative enough will yield too many false positives, which result in several negative financial outcomes as well. 

For passengers, these include:
* making unnecessary lodging and food accomodations 
* unnecessarily rescheduling connection flights

For airlines, these false positives could result in 
* unneeded overhead budgets for employee meals and lodging
* unnecessary overhead budgets for standby mechanic teams
* loss of customer goodwill that could be comparable to that resulting from an unforeseen flight delay (a false negative)

Our model balances avoiding the high cost of unanticipated delays with preventing the costs associated with false positives. 

Given that each airline likely has varying levels of exposure to these risks, it would also make sense for them to adjust the prediction thresholds of a model accordingly. A larger airline likely benefits from being more flexible with operational changes (more staff, more available gates in their terminal, etc.) and could lower their probability threshold to absorb the cost of additional false positives. A smaller airline would need to be more conservative and would therefore benefit from adjusting their threshold above 0.5, which is our model's default. Below we show the different evaluation metrics associated with different probability thresholds to help visualize these differences.

<img src="files/shared_uploads/hassansaadca@berkeley.edu/threshold_tuning.png" width = "600"/>


Adding to this, its likely that all models would yield varying performance metrics across different airports just as ours did, an example of which is shown in the image below. The left correlation matrix shows the distribution of predictions for JetBlue Airways for which there is a very high false positive rate. This means that JetBlue's implementation of our model could also consist of a final step to increase the probability threshold. Delta Airlines (on the right) could benefit from doing the opposite as a result of having a very low true positive rate.


<img src="files/shared_uploads/hassansaadca@berkeley.edu/airlines.png" width = "600"/>

This strategy is of course not limited to just airlines. Larger or smaller airports could benefit from doing the same, also based on their ability to absorb costs associated with different classifications. While the model achieves a balance across a wide range of different categories, it would definitely be useful to apply it in different manners depending on the situation at hand.

### Variable-Level Gap Analysis

To understand the strengths and areas for improvement in our final model, we performed a gap analysis to assess its performance in specific categories. Our approach was to evaluate performance in the following 3 categories:

- Time: `QUARTER`, `MONTH`, `DAY_OF_WEEK`
- Weather: `VIS`, `AIR_TMP` (bucketed to create binary variables)
- Flight/Airport Details : `ORIG_FPD`, `DEST_FPD`, `OP_CARRIER`, `DISTANCE_GROUP` (all except carrier are bucketed)

Overall, the gaps in our model performance are tied to segments with above-average class imbalances, exacerbated by our choice to optimize for F2. The model did not perform as well on smaller airports or on carriers with very low delay rates.  

Carrier, month, visibility, temperature, and departing flights per day (`ORIG_FPD`) had significant levels of differentiation in delay rates between buckets. These features had strong relationships with delays but the effects were muted by class imbalances - for example, visibility had 93% recall for the positive class, but precision was still only 38%.

### Future Work

There are three main areas where we feel like we could have improved our work:

1. **Caching and persisting dataframes more effectively:** Our performance times rank somewhere in the middle among the current ones that have been reported in the project leaderboard. Some teams took over 5 hours to tune a logistic regression model with 6 parameter combinations, where our baseline logistic regression model took under 10 minutes to train. On the other hand, running a Gradient Boost cross validation took us about 10 hours for a set of 16 parameter combinations. This isn't terrible, but there were some teams that were able to train GB models in ~15 minutes per their midterm presentations. We did consider using .csv files in our blob storage to leverage the row structure that each model relies on (vs. column structure of parquet files). However, there were several transformations we had to make to the dataframes as a part of our pipeline, which would have been very slow afer reading them in from .csv format. Finding a way to improve our training time would obviously increase our ability to quickly tune hyperparameters and output optimal models for testing on new data.

2. **Reduce the time for the data join**: In a real setting, we'd be running the join very often in order to make use of the most recent data. Touching on what was mentioned earlier in the feature engineering section, there are several features that are built by comparing one record for a specific tail number to the record just before it. This being said, it would be impossible to build these features if we were using batches of train data. Using batches would mean that flights for which we want to predict delays might not have a corresponding previous record in the training dataset. We would instead have to stream the data and run the join several hundred times a day. This would be impossible to achieve if we completed the join in the same way without constraining the records by a time range in addition to location. 

3. **Addressing class imbalances**: We balanced classes in our training dataset when tuning hyperparameters and when training our final model, but our model still had difficulty with the imbalanced classes in the validation datasets. We tested versions of the XGBoost model with varying values for `scale_pos_weight` to account for the uneven split, and although this improved our recall to up to 98% and boosted our F2 score, it decreased precision and accuracy to levels that we did not feel were acceptable. Finding a parameter that would tune the model to better address these imbalances without sacrificing precision and accuracy would greatly increase the value of our model.

## 8. Discussion on W261 Course Concepts

### Algorithms

**Decision Trees**

Decision trees form the basis for the Gradient and XGBoost models that we used on our final dataset. They can be used to solve both classification and regression problems, and they're good at dealing with different kinds of data input without relying on too many preprocessing techniques. 

At a high level, decision trees repeatedly split data into different buckets called nodes, with the goal of achieving a higher purity of class labels with each split. 

For classification problems like ours, entropy is used in order to calculate the purity, and is given by the equation \\(E = -[p_0 \times log(p_0) + p_1 \times log(p_1)]\\), where \\(p_m\\) represents the proportion of values that consist of label m in a specific node.

Information Gain (IG) represents the difference between the entropy of a parent node and the sum of weighted entropies of its children nodes. In the case of splitting a parent node into two children nodes, we implement the equation below. The higher the IG value, the better the choice of split.

\\(IG = Entropy(N_{parent})- [p_{child_1} \times Entropy(N_{child_1})+p_{child_2} \times Entropy(N_{child_2})]\\) 

A greedy decision tree algorithm will consider a split at every distinct value of every feature, and will select the one that corresponds to the highest information gain. Some common ways to expand on a decision tree include:
* Bagging: This entails building many decision trees using samples of the training data for each
* Random Forests: This method also relies on many decision trees using different samples of data, but each tree uses a different subset of the features. This helps prevent overfitting our overall model.

Both the Gradient Boost and XGBoost models that we built used a Pyspark Pipeline that is characteristic of decision tree models. We used one-hot encoding and bucketing transformations, but we didn't need to implement any standard scaling like a logistic regression model would require. 

**Gradient Boosting**

Like Bagging and Random Forest methods, Gradient Boosting also builds a model out of several different trees, and it relies on entropy and information gain for split selection as well. But where Boosting and Random Forests build these trees simultaneously and base predictions on the majority, a Gradient Boost model builds trees iteratively. This can be explained by the equation:

\\(\hat{f}(x) \leftarrow \hat{f}(x) + \nu \times \gamma \\) 

where

* \\(\hat{f}(x)\\) represents the current iteration's prediction 
* \\(\nu\\) represents a learning rate, which we tune as one of our hyperparameters
* \\(\gamma\\) represents the aggregate residual for each leaf node of our most recent decision tree 


The steps are:
1. Assign the same prediction to every record, \\(\hat{f}(x) = \frac{e^{log(odds)}}{1+e^{log(odds)}}\\)
2. Calculate the residual between the true label and our current prediction
3. Build trees or stumps for which the leaf nodes contain the residuals from step 2
4. Aggregate the residuals in each leaf (in regression, we use the mean, but see the toy notebook for how we do this with classification)
5. To each records prediction, add the aggregate residual scaled by the learning rate.
6. Repeat steps 2-6


**XGBoost**

The XGBoost process also uses multiple trees in sequence, but it adds a regularization element to the loss function with each iteration. 

The total loss function represented by \\(L(\hat{f}(x), y)\\) in a gradient boost model would be updated to \\(L(\hat{f}(x), y) + \lambda \times \gamma^2 + \alpha \times \lvert \gamma \rvert\\)


Similarly to regularization in logistic or linear regression, \\(\lambda \times \gamma^2\\) represents ridge or L2 regularization, but instead of passing in the weight vector, the loss is adjusted by the aggregate residual \\(\gamma\\), scaled by a hyperparameter we define, \\(\lambda\\). In our XGBoost model this is the `reg_lambda` parameter. The second portion, \\(\alpha \times \lvert \gamma \rvert\\) represents L1 or Lasso regularization. \\(\alpha\\) here is of course the `reg_alpha` parameter we tuned in our XGBoost cross validation process above. XGBoost and the regularization process can help us adjust for any potential overfitting that might be present in a GB model.

### Regularization
Overfitting and underfitting are two primary factors that lead to poor machine learning performance. Overfitting occurs when a model's performance doesn't generalize well to unseen data (i.e. our model suffers from high variance).

Regularization is a common tool that can help minimize the effects of overfitting by reducing the size of the weight coefficients in a linear model (and as we saw just above, it can also apply to non-linear models).

There are two main kinds of regularization, and both entail adding an additional factor to our loss function:

* L2 (Ridge) regularization adds the sum of squares of our weight components, scaled by a parameter \\(\lambda\\) which we have the ability to define when building our model. Note that the summation is across all the weight components for our additional term, where the original loss function is summed across all records in our training dataset. This shrinks our weight components down close to zero with higher values of \\(\lambda\\), minimizing the effect they have on our prediction.

$$
Loss Function = \frac{1}{N}\sum_{i=1}^{N}{(\hat{Y}-Y)^2}+\lambda\sum_{j=1}^{N}\theta_{j}^{2}
$$


- L1 (Lasso) regularization behaves in a similar way, but with higher values of \\(\lambda\\), weight coefficients can shrink down to zero. This effectively allows for Lasso regularization to perform feature selection. 

$$
Loss Function = \frac{1}{N}\sum_{i=1}^{N}{(\hat{Y}-Y)^2}+\lambda\sum_{j=1}^{N}|\theta_{j}|
$$

### One-hot encoding
Features could generally be classified as categorical or numeric values. For instance, a categorical feature "rank" could have a set of discrete values such as "low", "medium" and "high". Most machine learning algorithms require features to be in numerical form. 


To turn this categorical value into numeric, there are two main methods: integer encoding and one-hot encoding. Integer encoding, as its name suggest, works by assigning a number to each category. For example, 1 for "low", 2 for "medium" and 3 for "high". While integer encoding works, it imposes an order between categories that are ordinal in nature. This could lead to unexpected results or poor performance. One-hot encoding avoids this problem by creating \\( n-1\\) binary variables for the \\(n\\) values in the category. The downside of one-hot encoding is that, although it creates a sparse representation, it also rapidly expands the feature space, leading to high dimensionality.

For the one-hot encoding implemented in this project, it works like the below:
- "low" would be represented by 1, 0
- "medium" would be represented by 0, 1
- "high" would be represented by 0, 0

Some of the categorical values in the feature set of this project include: 
- `ORIGIN` (i.e. origin airport)
- `DEST` (i.e. destination airport)
- `OP_CARRIER` (i.e. airline code)
- `PRIOR_ARR_DEL` (i.e. indicator of previous flight's arrival delay)
- `PRIOR_DEP_DEL` (i.e. indicator of previous flight's departure delay)

The full list of categorical features could be found in the data pipeline notebook. For the one-hot encoding implementation, OneHotEncoder() from the PySpark MLlib was used.

## 9. References
**External Resources**


1. [Statista - Total operating revenue streams of U.S. airlines from 2004 to 2020](https://www.statista.com/statistics/197680/total-operating-revenues-in-us-airline-industry-since-2004/)
2. [FAA - Aviation Data Statistics - 2019 Cost of Delay Estimates](https://www.faa.gov/data_research/aviation_data_statistics/media/cost_delay_estimates.pdf)
3. [FAA - FAQ: Weather Delay](https://www.faa.gov/nextgen/programs/weather/faq/)  
4. [USDT - Bureau of Transportation Statistics](https://www.transtats.bts.gov/homepage.asp)
5. [NOAA - Quality Controlled Local Climatological Data](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00679)
6. [The Global Airport Database](https://www.partow.net/miscellaneous/airportdatabase/index.html)
7. [Federal Climate Complex Data Documentation for Integrated Surface Data (ISD)](https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf)
8. [Openflights.org](https://openflights.org/data.html) 
9. [Zheng, Casari: Feature Engineering for Machine Learning](https://www.repath.in/gallery/feature_engineering_for_machine_learning.pdf)


**Databricks Notebooks**

- [NB0 | Main DataBricks Notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574710004345/command/4070574710009557)
- [NB1 | Airline Weather Full Join](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102379377/command/1858507102379379)
- [NB2 | EDA Part 1 - Airline and Weather Data](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102378189/command/1858507102378190)
- [NB3 | EDA Part 2 - Weather and Time-Related Data](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102431022/command/1858507102431092)
- [NB4 | Feature Engineering](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102383058/command/1858507102383069)
- [NB5 | Machine Learning Pipeline](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574710015602/command/4070574710015603) 
- [NB6 | Decision Tree and Gradient Boost Toy](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102378262/command/1858507102378263)
- [NB7 | Logistic Regression Toy](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102371728/command/1858507102371729)
- [NB8 | Final Pipeline Test](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102387041/command/1858507102400388)
- [NB9 | Gap Analysis](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1731553363147681/command/1731553363147686)